In [1]:
import pandas as pd

# CSV-Datei laden
file_name = "rom_korr_full_website.csv"  # Ersetze "deine_datei.csv" durch den tatsächlichen Dateinamen
rom_korr_full_website = pd.read_csv(file_name)

# Die ersten 5 Zeilen anzeigen, um sicherzustellen, dass die Daten korrekt geladen wurden
print(rom_korr_full_website.head())

                           Date                       Sender  \
0   Donnerstag,  6. Januar 1791  August Wilhelm von Schlegel   
1  Mittwoch, 20. September 1797  August Wilhelm von Schlegel   
2         Samstag, 26. Mai 1798  August Wilhelm von Schlegel   
3    Mittwoch, 31. Oktober 1798  August Wilhelm von Schlegel   
4           [Mitte August 1801]             Sophie Bernhardi   

                     Recipient Place of Dispatch Place of Destination  \
0      Christian Gottlob Heyne         Göttingen            Göttingen   
1      Christian Gottlob Heyne              Jena            Göttingen   
2        Georg Joachim Göschen            Berlin              Leipzig   
3        Georg Joachim Göschen              Jena              Leipzig   
4  August Wilhelm von Schlegel            Berlin                 Jena   

                  Dispatch_GeoNames              Destination_GeoNames  \
0  https://www.geonames.org/2918632  https://www.geonames.org/2918632   
1  https://www.geonames.org/28

In [2]:
print(rom_korr_full_website.count())

Date                    4388
Sender                  4388
Recipient               4388
Place of Dispatch       4388
Place of Destination    4388
Dispatch_GeoNames       4285
Destination_GeoNames    4317
link                    4388
dtype: int64


In [4]:
#pip install folium geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import folium
import time
from geopy.geocoders import Nominatim

In [12]:
import requests
import pandas as pd
import numpy as np

# Zum Berechnen der Distanz
from geopy.distance import geodesic

# Für den Fallback-Geocoder
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

###############################################################################
#                               HILFSFUNKTIONEN
###############################################################################

def parse_geonames_id(gn_url):
    """
    Nimmt eine GeoNames-URL (z. B. "https://www.geonames.org/2918632")
    und gibt die ID (z. B. "2918632") zurück.
    Falls gn_url None oder kein GeoNames-Link ist, None zurück.
    """
    if not isinstance(gn_url, str) or "geonames.org/" not in gn_url:
        return None
    return gn_url.strip("/").split("/")[-1]

def geocode_with_geonames(geoname_id, geonames_username="obo000"):
    """
    Fragt die GeoNames-API mit einer geoNameId ab und gibt (lat, lon) zurück.
    Beispiel-URL:
      https://api.geonames.org/getJSON?geonameId=2918632&username=obo000
    Wenn es fehlschlägt oder lat/lng nicht vorhanden sind: (None, None).
    """
    if not geoname_id:
        return None, None

    url = f"http://api.geonames.org/getJSON?geonameId={geoname_id}&username={geonames_username}"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            data = r.json()
            lat = data.get("lat")
            lng = data.get("lng")
            if lat and lng:
                return float(lat), float(lng)
    except Exception as e:
        print(f"GeoNames Error (ID={geoname_id}): {e}")
    return None, None

# Nominatim-Geocoder für den Fallback
geolocator = Nominatim(user_agent="romantik_fallback_geocoder")
# RateLimiter, um nicht zu schnell auf den Server zuzugreifen
geocode_nominatim = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def geocode_with_nominatim(place_name):
    """
    Nimmt einen Ort als String (z. B. "Berlin") und gibt (lat, lon) via Nominatim zurück.
    Wenn nicht gefunden: (None, None).
    """
    if not isinstance(place_name, str) or not place_name.strip():
        return None, None
    try:
        location = geocode_nominatim(place_name)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Nominatim-Error für {place_name}: {e}")
    return None, None

###############################################################################
#                         KOORDINATEN + DISTANZ BERECHNEN
###############################################################################

def fill_coordinates_with_fallback(df):
    """
    Geht Zeile für Zeile durch:
      1) Versucht, aus Dispatch_GeoNames eine ID zu parsen und Koordinaten via GeoNames abzurufen.
         Wenn das nicht klappt (oder URL fehlt), nimmt Place of Dispatch und versucht Nominatim.
      2) Dasselbe für Destination_GeoNames / Place of Destination.
    
    Schreibt Koordinaten in Dispatch_Lat, Dispatch_Lon, Destination_Lat, Destination_Lon.
    """

    df["Dispatch_Lat"] = np.nan
    df["Dispatch_Lon"] = np.nan
    df["Destination_Lat"] = np.nan
    df["Destination_Lon"] = np.nan

    for idx, row in df.iterrows():
        # -----------------------------------------
        # 1) DISPATCH (Absendeort)
        # -----------------------------------------
        gn_url_disp = row["Dispatch_GeoNames"]        # z. B. "https://www.geonames.org/2895044"
        place_disp = row["Place of Dispatch"]         # z. B. "Jena"
        dispatch_id = parse_geonames_id(gn_url_disp)  # z. B. "2895044"

        d_lat, d_lon = None, None

        # Erst GeoNames
        if dispatch_id:
            d_lat, d_lon = geocode_with_geonames(dispatch_id, geonames_username="obo000")

        # Fallback: Nominatim
        if not d_lat or not d_lon:
            d_lat, d_lon = geocode_with_nominatim(place_disp)

        # Speichern in DF
        df.loc[idx, "Dispatch_Lat"] = d_lat
        df.loc[idx, "Dispatch_Lon"] = d_lon

        # -----------------------------------------
        # 2) DESTINATION (Empfangsort)
        # -----------------------------------------
        gn_url_dest = row["Destination_GeoNames"]
        place_dest = row["Place of Destination"]
        destination_id = parse_geonames_id(gn_url_dest)

        dest_lat, dest_lon = None, None

        # Erst GeoNames
        if destination_id:
            dest_lat, dest_lon = geocode_with_geonames(destination_id, geonames_username="obo000")

        # Fallback: Nominatim
        if not dest_lat or not dest_lon:
            dest_lat, dest_lon = geocode_with_nominatim(place_dest)

        # Speichern in DF
        df.loc[idx, "Destination_Lat"] = dest_lat
        df.loc[idx, "Destination_Lon"] = dest_lon

    return df

def compute_distances(df):
    """
    Berechnet für jede Zeile die Distanz (in km) zwischen
    Dispatch_(Lat,Lon) und Destination_(Lat,Lon).
    Speichert Ergebnis in der Spalte Distance_km.
    """
    df["Distance_km"] = np.nan

    for idx, row in df.iterrows():
        lat1 = row["Dispatch_Lat"]
        lon1 = row["Dispatch_Lon"]
        lat2 = row["Destination_Lat"]
        lon2 = row["Destination_Lon"]

        # Koordinaten-Check
        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            df.loc[idx, "Distance_km"] = geodesic((lat1, lon1), (lat2, lon2)).km

    return df

###############################################################################
#                            HAUPTPROGRAMM
###############################################################################

if __name__ == "__main__":
    # Angenommen, du hast deinen DataFrame schon eingelesen:
    df = pd.read_csv("rom_korr_full_website.csv")
    # Oder du definierst ihn an anderer Stelle.

    # 1) Koordinaten auffüllen (GeoNames => Fallback Nominatim)
    rom_korr_full_website = fill_coordinates_with_fallback(rom_korr_full_website)

    # 2) Distanz berechnen
    rom_korr_full_website = compute_distances(rom_korr_full_website)

    # Beispielausgabe
    print(rom_korr_full_website[[
        "Date", "Sender", "Recipient",
        "Place of Dispatch", "Dispatch_Lat", "Dispatch_Lon",
        "Place of Destination", "Destination_Lat", "Destination_Lon",
        "Distance_km"
    ]])




                               Date                           Sender  \
0       Donnerstag,  6. Januar 1791      August Wilhelm von Schlegel   
1      Mittwoch, 20. September 1797      August Wilhelm von Schlegel   
2             Samstag, 26. Mai 1798      August Wilhelm von Schlegel   
3        Mittwoch, 31. Oktober 1798      August Wilhelm von Schlegel   
4               [Mitte August 1801]                 Sophie Bernhardi   
...                             ...                              ...   
4464           [vor dem 16.04.1795]      Gottfried Philipp Michaelis   
4465      vor dem 10. Dezember 1801  Johann Friedrich Gottlieb Unger   
4466    Mittwoch,  9. Dezember 1801                     Ludwig Tieck   
4467  Donnerstag, 10. Dezember 1801                     Ludwig Tieck   
4468      vor dem 17. Dezember 1801                     Ludwig Tieck   

                             Recipient Place of Dispatch  Dispatch_Lat  \
0              Christian Gottlob Heyne         Göttingen     

In [13]:
#In CSV speichern
rom_korr_full_website.to_csv("rom_korr_full_website_coords.csv", index=False)

In [14]:
display(rom_korr_full_website)

,Date,Sender,Recipient,Place of Dispatch,Place of Destination,Dispatch_GeoNames,Destination_GeoNames,link,Dispatch_Lat,Dispatch_Lon,Destination_Lat,Destination_Lon,Distance_km
0,"Donnerstag, 6. Januar 1791",August Wilhelm von Schlegel,Christian Gottlob Heyne,Göttingen,Göttingen,https://www.geonames.org/2918632,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/1?...,51.534430,9.932280,51.534430,9.932280,0.000000
1,"Mittwoch, 20. September 1797",August Wilhelm von Schlegel,Christian Gottlob Heyne,Jena,Göttingen,https://www.geonames.org/2895044,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/2?...,50.928780,11.589900,51.534430,9.932280,133.955266
2,"Samstag, 26. Mai 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Berlin,Leipzig,https://www.geonames.org/2950159,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/3?...,52.524370,13.410530,51.339620,12.371290,149.951785
3,"Mittwoch, 31. Oktober 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Jena,Leipzig,https://www.geonames.org/2895044,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/4?...,50.928780,11.589900,51.339620,12.371290,71.276442
4,[Mitte August 1801],Sophie Bernhardi,August Wilhelm von Schlegel,Berlin,Jena,https://www.geonames.org/2950159,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/5?...,52.524370,13.410530,50.928780,11.589900,217.565424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4464,[vor dem 16.04.1795],Gottfried Philipp Michaelis,Caroline von Schelling,"Harburg, Elbe",Braunschweig,https://www.geonames.org/2910685,https://www.geonames.org/2945024,https://briefe-der-romantik.de/letters/view/19...,53.511750,9.815511,52.264658,10.523607,146.735939
4465,vor dem 10. Dezember 1801,Johann Friedrich Gottlieb Unger,Ludwig Tieck,Berlin,Dresden,https://www.geonames.org/2950159,https://www.geonames.org/2935022,https://briefe-der-romantik.de/letters/view/20...,52.510885,13.398937,51.049329,13.738144,164.293236
4466,"Mittwoch, 9. Dezember 1801",Ludwig Tieck,Carl Friedrich Ernst Frommann,Dresden,Jena,https://www.geonames.org/2935022,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/20...,51.049329,13.738144,50.928172,11.587936,151.571062
4467,"Donnerstag, 10. Dezember 1801",Ludwig Tieck,Johann Friedrich Gottlieb Unger,Dresden,Berlin,https://www.geonames.org/2935022,https://www.geonames.org/2950159,https://briefe-der-romantik.de/letters/view/20...,51.049329,13.738144,52.510885,13.398937,164.293236


In [ ]:
###### darunter der Code inklusive Cache: Schon gefundene Koordinaten werden gespeichert und wiederverwendet. Das reduziert die Laufzeit des Skriptes ungemein.

In [4]:
import requests
import pandas as pd
import numpy as np

# Zum Berechnen der Distanz
from geopy.distance import geodesic

# Für den Fallback-Geocoder
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

###############################################################################
#                               CACHE
###############################################################################

# Cache-Dictionaries für GeoNames und Nominatim
geonames_cache = {}
nominatim_cache = {}

###############################################################################
#                               HILFSFUNKTIONEN
###############################################################################

def parse_geonames_id(gn_url):
    """
    Nimmt eine GeoNames-URL (z. B. "https://www.geonames.org/2918632")
    und gibt die ID (z. B. "2918632") zurück.
    Falls gn_url None oder kein GeoNames-Link ist, None zurück.
    """
    if not isinstance(gn_url, str) or "geonames.org/" not in gn_url:
        return None
    return gn_url.strip("/").split("/")[-1]

def geocode_with_geonames(geoname_id, geonames_username="obo000"):
    """
    Fragt die GeoNames-API mit einer geoNameId ab und gibt (lat, lon) zurück.
    Wenn es fehlschlägt oder lat/lng nicht vorhanden sind: (None, None).
    Nutzt einen lokalen Cache, um Mehrfach-Abfragen zu vermeiden.
    """
    if not geoname_id:
        return None, None
    
    # 1) Cache checken
    if geoname_id in geonames_cache:
        return geonames_cache[geoname_id]
    
    # 2) Falls nicht im Cache, API-Abfrage machen
    url = f"http://api.geonames.org/getJSON?geonameId={geoname_id}&username={geonames_username}"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            data = r.json()
            lat = data.get("lat")
            lng = data.get("lng")
            if lat and lng:
                # In Cache speichern
                geonames_cache[geoname_id] = (float(lat), float(lng))
                return geonames_cache[geoname_id]
    except Exception as e:
        print(f"GeoNames Error (ID={geoname_id}): {e}")
    
    # Falls alles fehlschlägt
    geonames_cache[geoname_id] = (None, None)  # Auch im Cache ablegen
    return None, None

# Nominatim-Geocoder für den Fallback
geolocator = Nominatim(user_agent="romantik_fallback_geocoder")
# RateLimiter, um nicht zu schnell auf den Server zuzugreifen
geocode_nominatim = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def geocode_with_nominatim(place_name):
    """
    Nimmt einen Ort als String (z. B. "Berlin") und gibt (lat, lon) via Nominatim zurück.
    Wenn nicht gefunden: (None, None).
    Nutzt einen lokalen Cache, um Mehrfach-Abfragen zu vermeiden.
    """
    if not isinstance(place_name, str) or not place_name.strip():
        return None, None

    # 1) Cache checken
    place_name_clean = place_name.strip().lower()
    if place_name_clean in nominatim_cache:
        return nominatim_cache[place_name_clean]

    # 2) Falls nicht im Cache, Nominatim-Abfrage
    try:
        location = geocode_nominatim(place_name_clean)
        if location:
            nominatim_cache[place_name_clean] = (location.latitude, location.longitude)
            return nominatim_cache[place_name_clean]
    except Exception as e:
        print(f"Nominatim-Error für {place_name}: {e}")
    
    # Falls nicht gefunden oder Error
    nominatim_cache[place_name_clean] = (None, None)
    return None, None

###############################################################################
#                         KOORDINATEN + DISTANZ BERECHNEN
###############################################################################

def fill_coordinates_with_fallback(df):
    """
    Geht Zeile für Zeile durch:
      1) Versucht, aus Dispatch_GeoNames eine ID zu parsen und Koordinaten via GeoNames abzurufen.
         Wenn das nicht klappt (oder URL fehlt), nimmt Place of Dispatch und versucht Nominatim.
      2) Dasselbe für Destination_GeoNames / Place of Destination.
    
    Schreibt Koordinaten in Dispatch_Lat, Dispatch_Lon, Destination_Lat, Destination_Lon.
    """

    df["Dispatch_Lat"] = np.nan
    df["Dispatch_Lon"] = np.nan
    df["Destination_Lat"] = np.nan
    df["Destination_Lon"] = np.nan

    for idx, row in df.iterrows():
        # -----------------------------------------
        # 1) DISPATCH (Absendeort)
        # -----------------------------------------
        gn_url_disp = row["Dispatch_GeoNames"]        # z. B. "https://www.geonames.org/2895044"
        place_disp = row["Place of Dispatch"]         # z. B. "Jena"
        dispatch_id = parse_geonames_id(gn_url_disp)  # z. B. "2895044"

        d_lat, d_lon = None, None

        # Erst GeoNames
        if dispatch_id:
            d_lat, d_lon = geocode_with_geonames(dispatch_id, geonames_username="obo000")

        # Fallback: Nominatim
        if not d_lat or not d_lon:
            d_lat, d_lon = geocode_with_nominatim(place_disp)

        # Speichern in DF
        df.loc[idx, "Dispatch_Lat"] = d_lat
        df.loc[idx, "Dispatch_Lon"] = d_lon

        # -----------------------------------------
        # 2) DESTINATION (Empfangsort)
        # -----------------------------------------
        gn_url_dest = row["Destination_GeoNames"]
        place_dest = row["Place of Destination"]
        destination_id = parse_geonames_id(gn_url_dest)

        dest_lat, dest_lon = None, None

        # Erst GeoNames
        if destination_id:
            dest_lat, dest_lon = geocode_with_geonames(destination_id, geonames_username="obo000")

        # Fallback: Nominatim
        if not dest_lat or not dest_lon:
            dest_lat, dest_lon = geocode_with_nominatim(place_dest)

        # Speichern in DF
        df.loc[idx, "Destination_Lat"] = dest_lat
        df.loc[idx, "Destination_Lon"] = dest_lon

    return df

def compute_distances(df):
    """
    Berechnet für jede Zeile die Distanz (in km) zwischen
    Dispatch_(Lat,Lon) und Destination_(Lat,Lon).
    Speichert Ergebnis in der Spalte Distance_km.
    """
    df["Distance_km"] = np.nan

    for idx, row in df.iterrows():
        lat1 = row["Dispatch_Lat"]
        lon1 = row["Dispatch_Lon"]
        lat2 = row["Destination_Lat"]
        lon2 = row["Destination_Lon"]

        # Koordinaten-Check
        if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
            df.loc[idx, "Distance_km"] = geodesic((lat1, lon1), (lat2, lon2)).km

    return df

###############################################################################
#                            HAUPTPROGRAMM
###############################################################################

if __name__ == "__main__":
    # Beispielhaft: CSV einlesen
    df = pd.read_csv("rom_korr_full_website.csv")
    
    # 1) Koordinaten auffüllen (GeoNames => Fallback Nominatim, mit Cache)
    df = fill_coordinates_with_fallback(df)

    # 2) Distanz berechnen
    df = compute_distances(df)

    # Beispielausgabe
    print(df[[
        "Date", "Sender", "Recipient",
        "Place of Dispatch", "Dispatch_Lat", "Dispatch_Lon",
        "Place of Destination", "Destination_Lat", "Destination_Lon",
        "Distance_km"
    ]])

    # Optional: in CSV speichern
    df.to_csv("rom_korr_full_website_coords.csv", index=False)


                               Date                           Sender  \
0       Donnerstag,  6. Januar 1791      August Wilhelm von Schlegel   
1      Mittwoch, 20. September 1797      August Wilhelm von Schlegel   
2             Samstag, 26. Mai 1798      August Wilhelm von Schlegel   
3        Mittwoch, 31. Oktober 1798      August Wilhelm von Schlegel   
4               [Mitte August 1801]                 Sophie Bernhardi   
...                             ...                              ...   
4383           [vor dem 16.04.1795]      Gottfried Philipp Michaelis   
4384      vor dem 10. Dezember 1801  Johann Friedrich Gottlieb Unger   
4385    Mittwoch,  9. Dezember 1801                     Ludwig Tieck   
4386  Donnerstag, 10. Dezember 1801                     Ludwig Tieck   
4387      vor dem 17. Dezember 1801                     Ludwig Tieck   

                             Recipient Place of Dispatch  Dispatch_Lat  \
0              Christian Gottlob Heyne         Göttingen     